# A Beginner's Guide to Perform Molecular Dynamics Simulation of wetting processes

Instructions for running a GROMACS simulation of a wetting process. These resources are intended for pedagogical purposes, and were designed for the undergraduate and third-cycle courses at Politecnico di Torino. 

Michele Pellegrino (michele.pellegrino@polito.it)

All rights reserved (2026)

## Motivation

The physics of wetting processes is of great interest for several industrial fields related to energy engineering:
- Hydrophobic and  superhydrophobic surfaces (liquid-infused, grafted) for hydrodynamic drag reduction [1];
- Nanoporous hydrophilic surfaces for energy production and storage [2]; 
- Bio-inpired smart surfaces (self cleaning, anti-fogging properties) [3];
- Controlled boiling and heat transfer [4].

<img src="images/droplet-rough.png" alt="droplet_rough" width="700"/>

Figure - Example of a MD simulation of wetting: a water droplet spreading onto a rough amorphous silica surface (credit: Michele Pellegrino, unpublished).

In this exercise we will simulate and analyze the simulations of a cylindrical water droplet spreading onto a FCC crystal (like aluminum). We will see how the **contact angle** of the droplet on the solid surface changes as we make the surface more hydrophilic.

We will work with a **quasi-two dimensional system**, i.e. with a rather low aspect ratio between the in-plane simulation box edge and the other two. Mind that you can have true two-dimensional simulations in Molecular Dynamics, provided that you accept to describe atoms like "rods" rather than "spheres".

In [ ]:
import nglview as ng
import MDAnalysis as mda

import os
import numpy as np
import utilities_wetting as uw
import matplotlib.pyplot as plt

As usual, make sure we are in the right folder (`wetting`).

In [ ]:
!pwd

In [ ]:
# Run this cell only once!
workdir = os.getcwd()
print("Work directory:",workdir)

## Wettability of flat surfaces

We will first look at the output of a wetting on a hydrophobic flat surface. 

In [ ]:
!tar -xvf flat.tar.gz
%cd {workdir}/flat/

This is the _reference_ configuration, which is also used as initial configuration. If one were to use positions restraints, this configuration would be passed to `gmx grompp` via the `-r` flag:

In [ ]:
!cat refconf.gro

In [ ]:
# NGLviewer is really bad for visualizing wetting simulations
view_ref = ng.show_structure_file("refconf.gro")
view_ref.add_representation('licorice',selection="SOL")
view_ref.add_representation('ball+stick',selection="SUB")
view_ref

In [ ]:
# Use VMD, if you have it installed
!vmd refconf.gro

Let's have a look at the topology and molecular dynamics parameters files:

In [ ]:
%cd {workdir}/flat/nvt-1kJmol/
!ls

In [ ]:
!cat topology.top

We are going to use periodic boundary conditions along $y$ and $x$, but not $z$. This works by effectively placing a Lennard-Jones wall at the edges of the simulation box in the $z$ direction. You need to specify the atom type for this wall (in this case `CUW`). Note in `[ nonbond_params ]` that the $\sigma$ parameter is negative: this is a workaround to make the wall _purely repulsive_ in GROMACS. The atoms of type `AL` are instead the "aluminum-like" atoms forming the substrate. They only interact with water with a Lennard-Jones potential and do not interact with other `AL` atoms.

Let'e inspect the `.mdp` file:

In [ ]:
!cat nvt.mdp

A few comment on molecular dynamics parameters:
- We set `pbc = xy` to only have periodic boundary conditions in $x$ and $y$. Note that in GROMACS you need to have p.b.c. at least in one direction, and there is no equivalent to the "free" b.c. of LAMMPS. The parameters `nwall`, `wall-atomtype`, `wall-type` and `wall-r-linpot` define the type of walls placed at the $z$ edges. Note that periodicity affects the calculation of long-range Coulomb interaction (`ewald-geometry = 3dc`).
- We set `DispCorr = no`. This goes back to the discussion we had at the end of the `biphase` exercise: there's no point in doing the dispersion correction in a system with vapour or vacuum! Also note that the barostat coupling time is set to $-1$, i.e. we are not controlling pressure.
- We are freezing the atoms of the surface (`freezegrps = SUB` and `freezedim = Y Y Y`). They will not move during the simulation. This saves performance but it is _not_ a good idea in general: you want solid and liquid to exchange momentum! A better idea would be to use position restraints (as in the `zeolite` exercise).

Let's visualize the simulation:

In [ ]:
# NGLviewer believes for some reason this system to be enourmous and
# I have found no way to fix it...
# -> probably due to residue numbering, I am not going to lose my sleep on it

u_1kJmol = mda.Universe('system.tpr', 'traj_comp.xtc')
view_1kJmol = ng.show_mdanalysis(u_1kJmol)
view_1kJmol.add_representation('licorice',selection="SOL")
view_1kJmol.add_representation('ball+stick',selection="SUB")
view_1kJmol

In [ ]:
# Use VMD, if you have it installed
!vmd traj_comp.xtc confout.gro

### Density map and contour fitting

Now we are going to extract the _density field_ of the droplet, averaged in time and in the in-plane direction ($y$ direction in this case). To do this, we are going to use `gmx densmap`:

In [ ]:
!gmx help densmap

Note that the default averaging plane is $z$, so we have to change that. Also, the default bin size is very small (don't know why).

In [ ]:
!echo SOL | gmx densmap -f traj_comp.xtc -s system.tpr -b 100 -od densmap.dat -aver y -bin 0.05

We are now going to use a few functions from `utilities_wetting.py` to read the output density file, fit the droplet contour and compute the contact angle.

In [ ]:
# PBC box dimension along x
Lx = 16.18400
# PBC box dimension along y
Lz = 12.00000
# Our guess on where the contact angle needs to be computed (more on that later)
z_sub = 2.6
# Density threshould to discriminate between liquid and vapour phase
dens_thresh = 40

In [ ]:
dat_file_name = "densmap.dat"
# Reading the density map and storing it into a numpy array
density = uw.read_density(dat_file_name)
# Determining the interface contour and the contact angle from the density map
xc, zc, R, x, z, X, Z, intf_contour = uw.contact_angle(density, Lx, Lz, dens_thresh, z_sub)

In [ ]:
# Drawing a circle
s = np.linspace(0,2*np.pi,250)
circle_x = xc + R*np.cos(s)
circle_z = zc + R*np.sin(s)

cmap = plt.colormaps['Blues']
fig = plt.figure()
pmesh = plt.pcolormesh(X, Z, density, cmap=cmap)
pcont = plt.plot(intf_contour[0,:], intf_contour[1,:], 'k-')
pwall = plt.plot(x, z_sub*np.ones_like(x), 'r-')
pcirc = plt.plot(circle_x, circle_z, 'r-')
cmap = fig.colorbar(pmesh)
plt.axis('scaled')
plt.xlim([0,Lx])
plt.ylim([0,Lz])
plt.xlabel('$x$ [nm]')
plt.ylabel('$z$ [nm]')
cmap.ax.set_ylabel(r'number density [nm$^{-3}$]',rotation=270,labelpad=20)
plt.show()

How do we choose _where_ to compute the contact angle?
- Short answer: there no correct choice, so just be _sufficiently close_ to the surface;
- Long answer: there are many different choices, pick the one that suits you the best. In this case I considered the **liquid density layering** as reference and picked as "wall location" the first density peak of water. This peak correponds to water molecules adsorbed on the surface, which are much less mobile than the ones above. Think: where would you place a _no slip_ boundary condition?

### Changing the liquid-solid interaction energy 

In [ ]:
%cd {workdir}/flat

In [ ]:
!mkdir nvt-3kJmol
!cp nvt-1kJmol/topology.top nvt-3kJmol/topology.top
!cp nvt-1kJmol/nvt.mdp nvt-3kJmol/nvt.mdp

<span style="color:red">**Execise** - Now **open** the `topology.top` file in `nvt-3kJmol` and **change** the $\epsilon$ Lennard-Jones parameter of the substrate atoms to **3 kJ/mol**. **Then**, run the simulation (same `.mdp` file) and proceed with the same analysis workflow.</span>

In [ ]:
%cd  nvt-3kJmol/
!cat topology.top

In [ ]:
!gmx grompp -p topology.top -f nvt.mdp -c ../nvt-1kJmol/confout.gro -o system.tpr

In [ ]:
!gmx mdrun -v -s system.tpr

In [ ]:
view_3kJmol = ng.show_structure_file("confout.gro")
view_3kJmol.add_representation('licorice',selection="SOL")
view_3kJmol.add_representation('ball+stick',selection="SUB")
view_3kJmol

In [ ]:
# If you have VMD
!vmd traj_comp.xtc confout.gro

In [ ]:
!echo SOL | gmx densmap -f traj_comp.xtc -s system.tpr -b 100 -od densmap.dat -aver y -bin 0.05

In [ ]:
dat_file_name = "densmap.dat"
density = uw.read_density(dat_file_name)
xc, zc, R, x, z, X, Z, intf_contour = uw.contact_angle(density, Lx, Lz, dens_thresh, z_sub)

In [ ]:
circle_x = xc + R*np.cos(s)
circle_z = zc + R*np.sin(s)

cmap = plt.colormaps['Blues']
fig = plt.figure()
pmesh = plt.pcolormesh(X, Z, density, cmap=cmap)
pcont = plt.plot(intf_contour[0,:], intf_contour[1,:], 'k-')
pwall = plt.plot(x, z_sub*np.ones_like(x), 'r-')
pcirc = plt.plot(circle_x, circle_z, 'r-')
cmap = fig.colorbar(pmesh)
plt.axis('scaled')
plt.xlim([0,Lx])
plt.ylim([0,Lz])
plt.xlabel('$x$ [nm]')
plt.ylabel('$z$ [nm]')
cmap.ax.set_ylabel(r'number density [nm$^{-3}$]',rotation=270,labelpad=20)
plt.show()

<span style="color:red">**Execise** - Repeat with $\epsilon=5$ kJ/mol and $\epsilon=7$ kJ/mol.</span>

### 5 kJ/mol

In [ ]:
%cd {workdir}/flat
!mkdir nvt-5kJmol
!cp nvt-3kJmol/topology.top nvt-5kJmol/topology.top
!cp nvt-3kJmol/nvt.mdp nvt-5kJmol/nvt.mdp

In [ ]:
%cd  nvt-5kJmol/
!cat topology.top

In [ ]:
# AFTER editing .top file...
!gmx grompp -p topology.top -f nvt.mdp -c ../nvt-3kJmol/confout.gro -o system.tpr
!gmx mdrun -v -s system.tpr

In [ ]:
!echo SOL | gmx densmap -f traj_comp.xtc -s system.tpr -b 100 -od densmap.dat -aver y -bin 0.05
dat_file_name = "densmap.dat"
density = uw.read_density(dat_file_name)
xc, zc, R, x, z, X, Z, intf_contour = uw.contact_angle(density, Lx, Lz, dens_thresh, z_sub)

While waiting for the next case to finish, have a look at the compute statistics:

In [ ]:
!tail -n 40 md.log

### 7 kJ/mol

In [ ]:
%cd {workdir}/flat
!mkdir nvt-7kJmol
!cp nvt-5kJmol/topology.top nvt-7kJmol/topology.top
!cp nvt-5kJmol/nvt.mdp nvt-7kJmol/nvt.mdp

In [ ]:
%cd  nvt-7kJmol/
!cat topology.top

In [ ]:
# After editing .top file...
!gmx grompp -p topology.top -f nvt.mdp -c ../nvt-5kJmol/confout.gro -o system.tpr
!gmx mdrun -v -s system.tpr

In [ ]:
!echo SOL | gmx densmap -f traj_comp.xtc -s system.tpr -b 100 -od densmap.dat -aver y -bin 0.05
dat_file_name = "densmap.dat"
density = uw.read_density(dat_file_name)
xc, zc, R, x, z, X, Z, intf_contour = uw.contact_angle(density, Lx, Lz, dens_thresh, z_sub)

<span style="color:red">**Exercise** - Plot $\epsilon_{sw}$ against $\theta_0$. NB: $\epsilon_{sw}$ is the interaction parameter between water and aluminum, generated automatically via a combining rule (which one?).</span>

In [ ]:
# Use this cell for plotting your results
epsilon_sw = ...
theta_0 = ...

fig = plt.figure()
plt.plot(epsilon_sw, theta_0, 'ko')
plt.xlabel(r'$\epsilon_{sw}$ [kJ/mol]')
plt.ylabel(r'$\theta_0$ [deg]')
plt.show()

### Energetic considerations

The Lennard-Jones potential is supposed to model **dispersion** interactions. These are a type of **Van der Waals** interactions, i.e. electrostatic interactions between atoms that are not charged (_why?_). Dispersion interactions are the weakest intermolecular interactions you can think of, at least in Molecular Dynamics. The most hydrophylic case we simulated had $\varepsilon_{sw}\simeq 2.13$ kJ/mol, which is at the edge of what may be considered physically realistic (it starts to be as strong as a dipole-dipole interaction). And metals are generally even _more_ hydrophilic!

<img src="images/interatomic-interactions.png" alt="interatomic_interactions" width="650"/>

Our results are both quantitatively reasonable... and completely wrong:
- Water-metal interactions _can_ indeed be that strong, in principle as strong as hydrogen bonds.
- However, water-metal interactions _cannot be described with a simple pair potential_ (like Lennard-Jones). Doing meaningful simulations of water adsorption on solids is very challenging [5]. Usually people either go back to DFT, or utilize reactive or neural-network potentials.

Water aside, it is well know that a pair potential is unable to express the elastic properties of solid metals. This is not really a problem for us, since have have frozed the substrate. If you ever decide to simulate solid metal crystals, use a different potential, such as the Embedded Atom Method [6].

### A bit of theory

The equilibrium contact angle of a liquid droplet on a flat surface is given by the **Young-Dupré equation**:

$$\cos\theta_0 = \frac{\gamma_{sv}-\gamma_{sl}}{\gamma_{lv}},$$

where $\gamma_{sv}$, $\gamma_{sl}$ and $\gamma_{lv}$ are respectively the specific surface free energy of the solid-vapour, solid-liquid and liquid-vapour interfaces.

Note that this equation makes sense only if $\gamma_{sv}-\gamma_{sl}\le\gamma_{lv}$ and $\gamma_{sv}-\gamma_{sl}\ge-\gamma_{lv}$. If these conditions hold, the system is in the _partial wetting_ state, and a finite contact angle exists. Otherwise, the system is in a _complete wetting_ state (i.e. the droplet becomes a film) or in a _non-wetting_ state (i.e. the droplet remains perfectly spherical).

The free energy of the liquid-vapour surafce is usually expressed simply as $\gamma$ and can be calculate using the Kirkwood-Buff formula (does it ring a bell?).

On the other hand, it is virtually _impossible_ to compute $\gamma_{sv}$ and $\gamma_{sl}$ separately. It is however possible to compute the _work of adhesion_ $W_a=\gamma+\gamma_{sv}-\gamma_{sl}$ of a liquid film on the solid surface by using **free energy calculations**, but it is far from trivial!

## Wettability of rough surfaces

Determining the wetting state of a droplet on a non-flat surface (either naturally rough or artificially nanostructured) is very complex and _usually_ doesn't boil down to a simple equation.

Generally, the following distinction is made:
- If the liquid completely permeates or invades the rough surface, then the droplet sits in what is called a _Wenzel_ state;
- If pockets of vapour/air/vacuum form in the nooks and crannies of the rough surface, then the droplet sits in what is called a _Cassie-Baxter_ state.

<img src="images/wenzel-cassie.png" alt="wenzel_cassie" width="650"/>

Source: [Wikipedia](https://en.wikipedia.org/wiki/Cassie%27s_law)

Unfortunately, not all wetting states can be categorized as either _Wenzel_ or _Cassie-Baxter_: many relevant states are neither of the two (hemiwicking, pseudo-partial wetting, mixed states, ...). Most importantly, some surfaces allow for multiple **metastable** wetting states [7].

We will now study the wetting state of a water droplet sitting on a surface made of aluminum FCC nanopillars (i.e. a variation on the previous 'flat' system). Let's see how the reference configuration looks like:

In [ ]:
%cd {workdir}
!tar -xvf pillars.tar.gz

In [ ]:
%cd {workdir}/pillars/
!ls

In [ ]:
!cat refconf.gro

In [ ]:
# TODO - output the full trajectory and fix the problem with the size
# NGLviewer is really bad for visualizing wetting simulations
view_ref = ng.show_structure_file("refconf.gro")
view_ref.add_representation('licorice',selection="SOL")
view_ref.add_representation('ball+stick',selection="SUB")
view_ref

In [ ]:
# If you have VMD installed
!vmd refconf.gro

Again, I have already run the simulation for $\epsilon=1$ kJ/mol, you just need to look at the result and analyze the trajectory:

In [ ]:
%cd {workdir}/pillars/nvt-1kJmol/
!ls

In [ ]:
u_1kJmol_rough = mda.Universe('system.tpr', 'traj_comp.xtc')
view_1kJmol_rough = ng.show_mdanalysis(u_1kJmol_rough)
view_1kJmol_rough.add_representation('licorice',selection="SOL")
view_1kJmol_rough.add_representation('ball+stick',selection="SUB")
view_1kJmol_rough

In [ ]:
# If you have VMD installed
!vmd traj_comp.xtc confout.gro

In [ ]:
!echo SOL | gmx densmap -f traj_comp.xtc -s system.tpr -b 100 -od densmap.dat -aver y -bin 0.05

In [ ]:
Lx = 16.18400
Lz = 12.00000
z_sub = 2.8
dens_thresh = 40

In [ ]:
dat_file_name = "densmap.dat"
density = uw.read_density(dat_file_name)
xc, zc, R, x, z, X, Z, intf_contour = uw.contact_angle(density, Lx, Lz, dens_thresh, z_sub)

In [ ]:
s = np.linspace(0,2*np.pi,250)
circle_x = xc + R*np.cos(s)
circle_z = zc + R*np.sin(s)

cmap = plt.colormaps['Blues']
fig = plt.figure()
pmesh = plt.pcolormesh(X, Z, density, cmap=cmap)
pcont = plt.plot(intf_contour[0,:], intf_contour[1,:], 'k-')
pwall = plt.plot(x, z_sub*np.ones_like(x), 'r-')
pcirc = plt.plot(circle_x, circle_z, 'r-')
cmap = fig.colorbar(pmesh)
plt.axis('scaled')
plt.xlim([0,Lx])
plt.ylim([0,Lz])
plt.xlabel('$x$ [nm]')
plt.ylabel('$z$ [nm]')
cmap.ax.set_ylabel(r'number density [nm$^{-3}$]',rotation=270,labelpad=20)
plt.show()

According to **Cassie-Baxter equation**, the contact angle on a surface with heterogeneous wettability can be computed from the combination of equilibrium contact angles on each surface type:

$$ 
\cos\theta_{CB} = \sum_{i}^{types}\alpha_i\cos\theta_0^i \; ,
$$

where $\alpha_i$ is the surface fraction of each component and $\theta_0^i$ is the equilibirum contact angle on a ideal flat surface of type $i$.

In this case, we have two components: solid aluminum and vapour (or rather vacuum):

$$ 
\cos\theta_{CB} = \alpha_s\cos\theta_0^s + \alpha_v\cos\theta_0^v = \alpha_s\cos\theta_0^f-\alpha_v \; ,
$$

where the last equality comes from the fact that the contact angle of a droplet on vapour is 180$^{\circ}$ and $\cos(180^{\circ})=-1$. By construction, $\alpha_v\simeq5/9$ and $\alpha_s\simeq4/9$; hence using the value of the equilibrium contact angle from the first part of the notebook:

$$ 
\cos\theta_{CB} \simeq \frac{4}{9}\cos(114.6^{\circ}) - \frac{5}{9} \simeq -0.74 \; ,
$$

which yields $\theta_{CB}\simeq137.9^{\circ}$. Not too shabby!

&#9888; **NB**: Cassie-Baxter law is supposed to work at the _macroscale_, where one can define the different surface phases as continua. There's no guarantee it holds down to the molecular scale! Moreover, the derivation and the application of the formula itself is controversial [8]. &#9888;

### Wetting transitions

Let's see what happens as we increase the wettability of the surface. As we did for the flat surface, let's increase the $\epsilon$ of aluminum to 3 kJ/mol:

In [ ]:
%cd {workdir}/pillars
!mkdir nvt-3kJmol
!cp nvt-1kJmol/topology.top nvt-3kJmol/topology.top
!cp nvt-1kJmol/nvt.mdp nvt-3kJmol/nvt.mdp

In [ ]:
# Modify the topology file...
%cd  nvt-3kJmol/
!cat topology.top

In [ ]:
!gmx grompp -p topology.top -f nvt.mdp -c ../nvt-1kJmol/confout.gro -o system.tpr

In [ ]:
!gmx mdrun -v -s system.tpr

In [ ]:
view_3kJmol_rough = ng.show_structure_file("confout.gro")
view_3kJmol_rough.add_representation('licorice',selection="SOL")
view_3kJmol_rough.add_representation('ball+stick',selection="SUB")
view_3kJmol_rough

In [ ]:
# If you have VMD installed
!vmd traj_comp.xtc confout.gro

We can observe that the droplet has transitioned from a Cassie-Baxter to a **Wenzel** state. The equation for obtaining the equilibrium contact angle in the Wenzel state is:

$$\cos\theta_W = r\cdot\cos\theta_0,$$

where $\theta_0$ is the Young-Dupré contact angle on a flat surface with the same chemical composition and $r$ is the roughness ratio:

$$r = \frac{\mbox{total area}}{\mbox{apparent area}}$$

Note that $r\ge1$ and therefore:
- If $\theta_0>90^{\circ}$, the surface becomes even more hydrophilic;
- If $\theta_0<90^{\circ}$, the surface becomes even more hydrophobic.

The second case however, is very unlikely: a naturally hydrophobic surface would induce a Cassie-Baxter state instead. At the same time, it's unlikely to find an hydrophilic surface in the Cassie-Baxter state, as it would transition to a Wenzel state (as you have seen). The likely scenario is rather the following:

<img src="images/cassie-baxter-wenzel-pillars.png" alt="cassie_baxter_wenzel_pillars" width="750"/>

By construction, the height of the pillars $h$ is the same as their width, while the distance $l=5h/4$:

$$r = \frac{2h+l}{h+l} = \frac{2+\frac{5}{4}}{1+\frac{5}{4}} = \frac{13}{9} \simeq 1.44.$$

So we could compute the Wenzel contact angle... but it would be pointless: the droplet is likely stuck in a metastable wetting state due to **contact line pinning** on defects, and is unable to spread to its equilibrium state [9].

Modeling how roughness affects the statics and dynamics of wetting is an active research field. Contact me if you want to learn more, and possibly collaborate!

## References
**[1]** S. Lepikko et al., _Droplet slipperiness despite surface heterogeneity at molecular scale_, Nat. Chem. vol. 16, pp. 506-513 (2024), https://doi.org/10.1038/s41557-023-01346-3 \
**[2]** X. Liu et al., _Generic air-gen effect in nanoporous materials for sustainable energy harvesting from air humidity_, Adv. Mater. p. e2300748 (2023), https://doi.org/10.1002/adma.202300748 \
**[3]** Z. Han et al., _Flourishing bioinspired antifogging materials with superwettability: Progresses and challenges_, Adv. Mater. vol. 30, no. 13, p. 1704652 (2018), https://doi.org/10.1002/adma.201704652 \
**[4]** L. Bureš and Y. Sato, _On the modelling of the transition between contact-line and microlayer evaporation regimes in nucleate boiling_, JFM 916:A53 (2021), https://doi.org/10.1017/jfm.2021.204 \
**[5]** A. Groß and S. Sakong, _Ab Initio Simulations of Water/Metal Interfaces_, Chem. Rev. 122, 12, 10746-10776 (2022), https://doi.org/10.1021/acs.chemrev.1c00679 \
**[6]** M.S. Daw, S.M. Foiles and M.I. Baskes, _The embedded-atom method: a review of theory and applications_, Mater. Sci. Eng. R Rep. vol.9 7-8, p.251-310 (1993), https://doi.org/10.1016/0920-2307(93)90001-U \
**[7]** M. Amabili et al., _Unraveling the Salvinia Paradox: Design Principles for Submerged Superhydrophobicity_, Adv. Mater. vol.2 14 1500248 (2015), https://doi.org/10.1002/admi.201500248 \
**[8]** L. Gao and T.J. McCarthy, _How Wenzel and Cassie Were Wrong_, Langmuir 23 (7), 3762-3765 (2007), https://doi.org/10.1021/la062634a \
**[9]** H. Perrin et al., _Defects at the Nanoscale Impact Contact Line Motion at all Scales_, Phys. Rev. Lett. 116, 184502 (2016), https://doi.org/10.1103/PhysRevLett.116.184502

# "One Last Thing"

## Additional GROMACS resources

- **[BioExcel GROMACS forum](https://gromacs.bioexcel.eu/)** - The best place to ask (relevant) questions about GROMACS and get in direct contact with the devlopers. Just follow the forum guidelines and be polite!
- **[BioExcel Youtube channel](https://www.youtube.com/@BioExcelCoE)** - Here you can find seminars and lectured given by GROMACS devlopers and contributors.
- **[Official GROMACS tutorials](https://tutorials.gromacs.org/)** - Tutorials developed internally by the GROMACS team, mostly about biochamistry and molecular biology.
- **Tutorials from the community** - I suggest having a look at material developed by the community, such as the 'historic' tutorials by J. Lemkul (http://www.mdtutorials.com/gmx/) and the ones by S. Gravelle (https://gromacstutorials.github.io/).
- **[GROMACS workshop](https://www.gromacs.org/workshop.html)** (discontinued?) - Workshop on learning how to contribute to the GROMACS code, given in 2023 and 2024. You can still find useful material, especially if you plan to 'hack' into the source code yourself.

## Alternatives to GROMACS

- **[LAMMPS](https://www.lammps.org/#nogo)** - The usual suspect when it comes to material science. More flexible at the price of being slower and less user-friendly.
- **[NAMD](https://www.ks.uiuc.edu/Research/namd/)** - It competes with GROMACS for the title of most popular Molecular Dynamics code. Mostly used for large biochemistry simulations.
- **[OpenMM](https://openmm.org/)** - The rising star of Neural Network Potentials. It achieve remarkable performance despite being Python-based.
- **[ASE](https://ase-lib.org/)** - Python-based simulation _orchestrator_, which allows to borrow kernels from other codes. Good for trying things out (e.g. Neural Network Potentials), not designed for efficiency and scalability.

And **many** more!